# Using Dask for splitting dataset

## Converting chunks to include row groups

In [1]:
import dask.dataframe as dd
import pandas as pd

In [2]:
import os
files = os.listdir('data')

# Filter the files to only include the parquet files on the form "chunk_xxx.parquet"
files = [f for f in files if f.startswith('chunk_') and f.endswith('.parquet')]
files


['chunk_114.parquet',
 'chunk_227.parquet',
 'chunk_327.parquet',
 'chunk_432.parquet',
 'chunk_558.parquet',
 'chunk_684.parquet',
 'chunk_784.parquet',
 'chunk_853.parquet',
 'chunk_last.parquet']

### 9th chunk

In [3]:
# First chunk
df = pd.read_parquet('data/' + files[8])
df

,id,content,type,title,authors,domain,url
0,9869341,"The colon cancer story began in 2011, when Dr....",reliable,Why Is This Bacterium Hiding in Human Tumors?,Gina Kolata,nytimes.com,https://www.nytimes.com/2017/11/23/health/bact...
1,9869342,This is highlighted in the grid by 17A’s FLIPP...,reliable,Snapping With Only Slight Provocation,Deb Amlen,nytimes.com,https://www.nytimes.com/2017/11/23/crosswords/...
2,9869343,"Born at the height of the Cold War, Pine Gap w...",reliable,An American Spy Base Hidden in Australia’s Out...,Jackie Dent,nytimes.com,https://www.nytimes.com/2017/11/23/world/austr...
3,9869344,OBITUARIES\n\nThe byline for an obituary on Th...,reliable,"Corrections: November 24, 2017",,nytimes.com,https://www.nytimes.com/2017/11/23/pageoneplus...
4,9869345,"“I want to be great, and I have confidence in ...",reliable,Vikings Top Lions and Tighten Grip on N.F.C. N...,The Associated Press,nytimes.com,https://www.nytimes.com/2017/11/23/sports/foot...
...,...,...,...,...,...,...,...
8951,9878292,"Photo\n\nMan does not live by bread alone, it ...",reliable,Corruption With a Chance of Meatballs,Clyde Haberman,nytimes.com,https://www.nytimes.com/2018/02/06/opinion/cor...
8952,9878293,"VIX, which measures investor expectations that...",reliable,"Volatility Rattles Stocks, and Investors Who B...",Landon Thomas Jr.,nytimes.com,https://www.nytimes.com/2018/02/06/business/st...
8953,9878294,Housing Authority residents in the audience we...,reliable,"As 4 of 5 in Public Housing Lost Heat, a Deman...",William Neuman,nytimes.com,https://www.nytimes.com/2018/02/06/nyregion/he...
8954,9878295,"He has a curious way of expressing it, though....",reliable,Who Said It: Pepé Le Pew or Donald Trump?,Jennifer Finney Boylan,nytimes.com,https://www.nytimes.com/2018/02/06/opinion/pep...


In [4]:
# Saves the dataframe as a parquet file. Remember to specify the row_group_size
df.to_parquet('data/c9.parquet', row_group_size=10000)

In [5]:
# Reads the parquet file. We can now use the dask dataframe to avoid loading the entire file into memory
ddf = dd.read_parquet('data/c9.parquet')
ddf

,id,content,type,title,authors,domain,url
npartitions=1,,,,,,,
,int64,object,object,object,object,object,object
,...,...,...,...,...,...,...


In [6]:
# Loading only 1 partition
ddf.partitions[0].compute()

,id,content,type,title,authors,domain,url
0,9869341,"The colon cancer story began in 2011, when Dr....",reliable,Why Is This Bacterium Hiding in Human Tumors?,Gina Kolata,nytimes.com,https://www.nytimes.com/2017/11/23/health/bact...
1,9869342,This is highlighted in the grid by 17A’s FLIPP...,reliable,Snapping With Only Slight Provocation,Deb Amlen,nytimes.com,https://www.nytimes.com/2017/11/23/crosswords/...
2,9869343,"Born at the height of the Cold War, Pine Gap w...",reliable,An American Spy Base Hidden in Australia’s Out...,Jackie Dent,nytimes.com,https://www.nytimes.com/2017/11/23/world/austr...
3,9869344,OBITUARIES\n\nThe byline for an obituary on Th...,reliable,"Corrections: November 24, 2017",,nytimes.com,https://www.nytimes.com/2017/11/23/pageoneplus...
4,9869345,"“I want to be great, and I have confidence in ...",reliable,Vikings Top Lions and Tighten Grip on N.F.C. N...,The Associated Press,nytimes.com,https://www.nytimes.com/2017/11/23/sports/foot...
...,...,...,...,...,...,...,...
8951,9878292,"Photo\n\nMan does not live by bread alone, it ...",reliable,Corruption With a Chance of Meatballs,Clyde Haberman,nytimes.com,https://www.nytimes.com/2018/02/06/opinion/cor...
8952,9878293,"VIX, which measures investor expectations that...",reliable,"Volatility Rattles Stocks, and Investors Who B...",Landon Thomas Jr.,nytimes.com,https://www.nytimes.com/2018/02/06/business/st...
8953,9878294,Housing Authority residents in the audience we...,reliable,"As 4 of 5 in Public Housing Lost Heat, a Deman...",William Neuman,nytimes.com,https://www.nytimes.com/2018/02/06/nyregion/he...
8954,9878295,"He has a curious way of expressing it, though....",reliable,Who Said It: Pepé Le Pew or Donald Trump?,Jennifer Finney Boylan,nytimes.com,https://www.nytimes.com/2018/02/06/opinion/pep...


## Testing on cluster for fun

In [3]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=8, threads_per_worker=1, memory_limit='2GB')
client = Client(cluster)
client

2023-03-08 12:14:12,164 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ag8njkiy', purging
2023-03-08 12:14:12,164 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-_tbv1twq', purging
2023-03-08 12:14:12,164 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-jaun1nko', purging
2023-03-08 12:14:12,165 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-71j00owi', purging
2023-03-08 12:14:12,165 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-4byx7pxr', purging
2023-03-08 12:14:12,165 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-xg4nlxok', purging
2023-03-08 12:14:12,165 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-kbzte9hk', purging

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34349,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:43665,Total threads: 1
Dashboard: http://127.0.0.1:36615/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:46563,


In [4]:
ddf_test = ddf.copy()
ddf_test

,id,content,type,title,authors,domain,url
npartitions=31,,,,,,,
,int64,object,object,object,object,object,object
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [5]:
# Split the dataframe into a test and a train set
ddf_test, ddf_train = ddf_test.random_split([0.2, 0.8], random_state=42)

In [6]:
print(len(ddf_test))

225944


In [7]:
print(len(ddf_train))

2023-03-08 12:15:17,499 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.20 GiB -- Worker memory limit: 1.86 GiB


904056


In [8]:
# Write the test and train set to parquet files
ddf_test.to_parquet('data/test.parquet', row_group_size=10000)

In [9]:
# Write the train set to parquet files
ddf_train.to_parquet('data/train.parquet', row_group_size=10000)

2023-03-08 12:16:12,972 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.31 GiB -- Worker memory limit: 1.86 GiB
2023-03-08 12:16:24,399 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.43 GiB -- Worker memory limit: 1.86 GiB
2023-03-08 12:16:38,115 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

In [10]:
train = dd.read_parquet('data/train.parquet')
train

,id,content,type,title,authors,domain,url
npartitions=31,,,,,,,
,int64,object,object,object,object,object,object
,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...
,...,...,...,...,...,...,...


In [11]:
# Split train into training and validation sets
train, val = train.random_split([0.8, 0.2], random_state=42)


In [15]:
# Barplot over the type column
train['type'].value_counts().compute()

2023-03-08 12:26:08,263 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.31 GiB -- Worker memory limit: 1.86 GiB
2023-03-08 12:26:10,178 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 1.50 GiB -- Worker memory limit: 1.86 GiB
2023-03-08 12:26:10,875 - distributed.worker.memory - WARNING - Worker is at 76% memory usage. Resuming worker. Process memory: 1.42 GiB -- Worker memory limit: 1.86 GiB
2023-03-08 12:26:11,236 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memo

political     187035
unreliable    110191
bias           94252
conspiracy     79451
fake           79257
rumor          44269
unknown        30372
               29644
hate           26430
junksci        14718
clickbait      13933
satire          9233
reliable        4188
Name: type, dtype: int64

2023-03-08 12:26:53,952 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.43 GiB -- Worker memory limit: 1.86 GiB
2023-03-08 12:27:03,952 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.43 GiB -- Worker memory limit: 1.86 GiB
2023-03-08 12:27:14,055 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

In [4]:
df['type'].value_counts()

political     292180
unreliable    172478
bias          147189
conspiracy    124418
fake          123966
rumor          69018
unknown        47600
               45988
hate           41404
junksci        23048
clickbait      21761
satire         14326
reliable        6624
Name: type, dtype: int64